In [1]:
import sklearn
print(sklearn.__version__)


1.5.0


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import warnings

In [2]:
df=pd.read_csv('finaldata.csv')
df

,Company,TypeName,Ram,Weight,Price,TouchScreen,Ips,ppi,Cpu Brand,HDD,SSD,Hybrid,Flash_Storage,Gpu brand,os
0,Apple,Ultrabook,8,1.37,71378.6832,0,1,226.983005,Intel Core i5,0,128,0,0,Intel,Mac
1,Apple,Ultrabook,8,1.34,47895.5232,0,0,127.677940,Intel Core i5,0,0,0,128,Intel,Mac
2,HP,Notebook,8,1.86,30636.0000,0,0,141.211998,Intel Core i5,0,256,0,0,Intel,Others/No Os/Linux
3,Apple,Ultrabook,16,1.83,135195.3360,0,1,220.534624,Intel Core i7,0,512,0,0,AMD,Mac
4,Apple,Ultrabook,8,1.37,96095.8080,0,1,226.983005,Intel Core i5,0,256,0,0,Intel,Mac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,Lenovo,2 in 1 Convertible,4,1.80,33992.6400,1,1,157.350512,Intel Core i7,0,128,0,0,Intel,Windows
1298,Lenovo,2 in 1 Convertible,16,1.30,79866.7200,1,1,276.053530,Intel Core i7,0,512,0,0,Intel,Windows
1299,Lenovo,Notebook,2,1.50,12201.1200,0,0,111.935204,Other Intel Processor,0,0,0,64,Intel,Windows
1300,HP,Notebook,6,2.19,40705.9200,0,0,100.454670,Intel Core i7,1000,0,0,0,AMD,Windows


In [3]:

df = df.rename(columns={'Cpu Brand': 'Cpu_Brand'})
df = df.rename(columns={'Gpu brand': 'Gpu_Brand'})
df = df.rename(columns={'HDD': 'Hdd'})
df = df.rename(columns={'SSD': 'Ssd'})
df.columns


Index(['Company', 'TypeName', 'Ram', 'Weight', 'Price', 'TouchScreen', 'Ips',
       'ppi', 'Cpu_Brand', 'Hdd', 'Ssd', 'Hybrid', 'Flash_Storage',
       'Gpu_Brand', 'os'],
      dtype='object')

In [4]:
sample = {'Company': ['HP'], 
          'TypeName':['Ultrabook'], 
          'Ram':[4],
          'Weight':[1.3],
          'TouchScreen':[0],
          'Ips':[1],
          'ppi':[226.983005], 
          'Cpu_Brand':['Intel Core i3'], 
          'Hdd':[0], 
          'Ssd':[128], 
          'Hybrid':[0], 
          'Flash_Storage':[0],
         'Gpu_Brand': ['Intel'],
         'os':['Windows']}
samp = pd.DataFrame(sample)
samp

,Company,TypeName,Ram,Weight,TouchScreen,Ips,ppi,Cpu_Brand,Hdd,Ssd,Hybrid,Flash_Storage,Gpu_Brand,os
0,HP,Ultrabook,4,1.3,0,1,226.983005,Intel Core i3,0,128,0,0,Intel,Windows


In [5]:
df['Hybrid'].unique()

array([   0, 1000,  508], dtype=int64)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        1302 non-null   object 
 1   TypeName       1302 non-null   object 
 2   Ram            1302 non-null   int64  
 3   Weight         1302 non-null   float64
 4   Price          1302 non-null   float64
 5   TouchScreen    1302 non-null   int64  
 6   Ips            1302 non-null   int64  
 7   ppi            1302 non-null   float64
 8   Cpu_Brand      1302 non-null   object 
 9   Hdd            1302 non-null   int64  
 10  Ssd            1302 non-null   int64  
 11  Hybrid         1302 non-null   int64  
 12  Flash_Storage  1302 non-null   int64  
 13  Gpu_Brand      1302 non-null   object 
 14  os             1302 non-null   object 
dtypes: float64(3), int64(7), object(5)
memory usage: 152.7+ KB


In [7]:
df['Hybrid'].value_counts()

Hybrid
0       1290
1000      11
508        1
Name: count, dtype: int64

In [8]:
df['target'] = df['Price']
df.drop(['Price'], axis=1, inplace=True)
df.head()

,Company,TypeName,Ram,Weight,TouchScreen,Ips,ppi,Cpu_Brand,Hdd,Ssd,Hybrid,Flash_Storage,Gpu_Brand,os,target
0,Apple,Ultrabook,8,1.37,0,1,226.983005,Intel Core i5,0,128,0,0,Intel,Mac,71378.6832
1,Apple,Ultrabook,8,1.34,0,0,127.677940,Intel Core i5,0,0,0,128,Intel,Mac,47895.5232
2,HP,Notebook,8,1.86,0,0,141.211998,Intel Core i5,0,256,0,0,Intel,Others/No Os/Linux,30636.0000
3,Apple,Ultrabook,16,1.83,0,1,220.534624,Intel Core i7,0,512,0,0,AMD,Mac,135195.3360
4,Apple,Ultrabook,8,1.37,0,1,226.983005,Intel Core i5,0,256,0,0,Intel,Mac,96095.8080


In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(['target'], axis=1)
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=10)


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# bagging

In [11]:
ncols =  X.select_dtypes(include=['int64','float64']).columns
ccols = X_train.select_dtypes(include=['object']).columns

In [12]:
ntransformer = Pipeline(steps=[
    ("imputeN",SimpleImputer(strategy='mean')),
    ("scale",StandardScaler())
])

In [13]:
ctransformer = Pipeline(steps=[
    ("imputeN",SimpleImputer(strategy='most_frequent')),
    ("encode",OneHotEncoder(handle_unknown='ignore'))
])

In [15]:
from sklearn.compose import ColumnTransformer
preprocessing = ColumnTransformer(transformers=[
    ('numerical',ntransformer,ncols),
    ('categorical',ctransformer,ccols)
])

In [16]:
#Lets get estimator first
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

#Create estimator
estimator = KNeighborsRegressor(n_neighbors=3, weights='distance', algorithm='kd_tree')

In [17]:
bagging = BaggingRegressor(estimator=estimator, n_estimators=10, max_samples=0.5)


In [18]:
#Now add estimator to pipeline
pipelinemodel = Pipeline(steps=[
    ('pre',preprocessing),
    ('est',bagging)
])


In [19]:
from sklearn import set_config
set_config(display='diagram')
pipelinemodel.fit(X_train, y_train)


Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputeN',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  Index(['Ram', 'Weight', 'TouchScreen', 'Ips', 'ppi', 'Hdd', 'Ssd', 'Hybrid',
       'Flash_Storage'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputeN',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Company', 'TypeName', 'Cpu_Brand', 'Gpu_Brand', 'os'], dtype='object'))])),
                ('est',
                 BaggingRegressor(estimator=KNeighborsRegressor(algorithm='kd_tree',
                                                                n_neighbors=3,
                                                                weights='distance'),
                                  max_samples=0.5))])

In [20]:
 ypred = pipelinemodel.predict(X_test)


In [21]:
res = pipelinemodel.predict(samp)
res

array([60143.08833244])

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


mae = mean_absolute_error(y_test, ypred)
mse = mean_squared_error(y_test, ypred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, ypred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

Mean Absolute Error: 12013.359658269423
Mean Squared Error: 298903311.99178666
Root Mean Squared Error: 17288.820433788613
R-squared: 0.7433268031123743


In [23]:
print(pipelinemodel.score(X_train, y_train))
print(pipelinemodel.score(X_test, y_test))

0.8969105334306878
0.7433268031123743


# Voting

In [24]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso


model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()
model4 = Lasso(alpha=0.1)
from sklearn.ensemble import VotingRegressor
ensemble = VotingRegressor(estimators=[('dt', model1), ('lr', model2), ('kn', model3),("la", model4)])


In [25]:
pipelinemodel1 = Pipeline(steps=[
    ('pre',preprocessing),
    ('est',ensemble)
])

In [26]:
from sklearn import set_config
set_config(display='diagram')
pipelinemodel1.fit(X_train, y_train)


C:\Users\kinle\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.297e+10, tolerance: 1.501e+08
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputeN',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  Index(['Ram', 'Weight', 'TouchScreen', 'Ips', 'ppi', 'Hdd', 'Ssd', 'Hybrid',
       'Flash_Storage'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputeN',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Company', 'TypeName', 'Cpu_Brand', 'Gpu_Brand', 'os'], dtype='object'))])),
                ('est',
                 VotingRegressor(estimators=[('dt', DecisionTreeRegressor()),
                                             ('lr', LinearRegression()),
                                             ('kn', KNeighborsRegressor()),
                                             ('la', Lasso(alpha=0.1))]))])

In [27]:
 ypred1 = pipelinemodel1.predict(X_test)


In [28]:
mae = mean_absolute_error(y_test, ypred1)
mse = mean_squared_error(y_test, ypred1)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, ypred1)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

Mean Absolute Error: 10595.222239400502
Mean Squared Error: 228735273.17474294
Root Mean Squared Error: 15123.996600592813
R-squared: 0.8035812536987249


In [29]:
print(pipelinemodel1.score(X_train, y_train))
print(pipelinemodel1.score(X_test, y_test))

0.8998074058500306
0.8035812536987249


# Stacking

In [30]:
from sklearn.ensemble import StackingRegressor
m1 = DecisionTreeRegressor()
m2 = KNeighborsRegressor()
meta = LinearRegression()
stack = StackingRegressor(estimators=[('dt',m1),('km',m2)], final_estimator=meta)


In [31]:
pipelinemodel2 = Pipeline(steps=[
    ('pre',preprocessing),
    ('est',stack)
])

In [32]:
from sklearn import set_config
set_config(display='diagram')
pipelinemodel2.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputeN',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  Index(['Ram', 'Weight', 'TouchScreen', 'Ips', 'ppi', 'Hdd', 'Ssd', 'Hybrid',
       'Flash_Storage'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputeN',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Company', 'TypeName', 'Cpu_Brand', 'Gpu_Brand', 'os'], dtype='object'))])),
                ('est',
                 StackingRegressor(estimators=[('dt', DecisionTreeRegressor()),
                                               ('km', KNeighborsRegressor())],
                                   final_estimator=LinearRegression()))])

In [33]:
 ypred2 = pipelinemodel2.predict(X_test)

In [34]:
mae = mean_absolute_error(y_test, ypred2)
mse = mean_squared_error(y_test, ypred2)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, ypred2)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

Mean Absolute Error: 11106.013453944104
Mean Squared Error: 261475371.95986018
Root Mean Squared Error: 16170.200121206299
R-squared: 0.7754667916487907


In [35]:
print(pipelinemodel2.score(X_train, y_train))
print(pipelinemodel2.score(X_test, y_test))

0.9353435091297613
0.7754667916487907


# Adaboost

In [38]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import Lasso
base_estimator = DecisionTreeRegressor(max_depth=5)
ada_boost = AdaBoostRegressor(estimator= base_estimator, n_estimators = 10, random_state = 1)

In [39]:
pipelinemodel3 = Pipeline(steps=[
    ('pre',preprocessing),
    ('est',ada_boost)
])

In [40]:
from sklearn import set_config
set_config(display='diagram')
pipelinemodel3.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputeN',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  Index(['Ram', 'Weight', 'TouchScreen', 'Ips', 'ppi', 'Hdd', 'Ssd', 'Hybrid',
       'Flash_Storage'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputeN',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Company', 'TypeName', 'Cpu_Brand', 'Gpu_Brand', 'os'], dtype='object'))])),
                ('est',
                 AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=5),
                                   n_estimators=10, random_state=1))])

In [41]:
 ypred3 = pipelinemodel3.predict(X_test)

In [42]:
mae = mean_absolute_error(y_test, ypred3)
mse = mean_squared_error(y_test, ypred3)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, ypred3)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)+
print("R-squared:", r2)

Mean Absolute Error: 13417.618642737472
Mean Squared Error: 317039216.9585812
Root Mean Squared Error: 17805.595102623814
R-squared: 0.727753202822508


In [43]:
print(pipelinemodel3.score(X_train, y_train))
print(pipelinemodel3.score(X_test, y_test))

0.839535963687277
0.727753202822508


In [44]:
import pickle

pickle.dump(pipelinemodel1, open('model.pkl', 'wb'))